In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

def detectar_lineas_hough(imagen_path, theta_res=0.1, rho_res=1, threshold=100, umbral_bajo=30, umbral_alto=150):
    # Cargar la imagen
    imagen = cv2.imread(imagen_path, cv2.IMREAD_GRAYSCALE)

    # Aplicar el filtro Gaussiano para reducir el ruido
    imagen_suavizada = cv2.GaussianBlur(imagen, (5, 5), 1.4)

    # Calcular los gradientes utilizando los kernels de Sobel
    Ix = cv2.Sobel(imagen_suavizada, cv2.CV_64F, 1, 0, ksize=3)
    Iy = cv2.Sobel(imagen_suavizada, cv2.CV_64F, 0, 1, ksize=3)

    # Calcular la magnitud y el ángulo del gradiente
    magnitud_gradiente = np.sqrt(Ix**2 + Iy**2)
    angulo_gradiente = np.arctan2(Iy, Ix)

    # Supresión no máxima
    imagen_suprimida = np.zeros_like(magnitud_gradiente)
    angulo_gradiente = angulo_gradiente * 180.0 / np.pi
    angulo_gradiente[angulo_gradiente < 0] += 180

    for i in range(1, imagen.shape[0] - 1):
        for j in range(1, imagen.shape[1] - 1):
            angulo = angulo_gradiente[i, j]
            if (0 <= angulo < 22.5) or (157.5 <= angulo <= 180):
                vecinos = [magnitud_gradiente[i, j - 1], magnitud_gradiente[i, j + 1]]
            elif 22.5 <= angulo < 67.5:
                vecinos = [magnitud_gradiente[i - 1, j - 1], magnitud_gradiente[i + 1, j + 1]]
            elif 67.5 <= angulo < 112.5:
                vecinos = [magnitud_gradiente[i - 1, j], magnitud_gradiente[i + 1, j]]
            else:
                vecinos = [magnitud_gradiente[i - 1, j + 1], magnitud_gradiente[i + 1, j - 1]]

            if magnitud_gradiente[i, j] >= max(vecinos):
                imagen_suprimida[i, j] = magnitud_gradiente[i, j]

    # Doble umbral
    imagen_umbral = np.zeros_like(imagen_suprimida)
    imagen_umbral[(imagen_suprimida >= umbral_bajo) & (imagen_suprimida <= umbral_alto)] = 255

    # Definir los parámetros de la transformada de Hough
    thetas = np.deg2rad(np.arange(-90, 90, theta_res))
    diag_len = int(np.sqrt(imagen_umbral.shape[0]**2 + imagen_umbral.shape[1]**2))
    rhos = np.arange(-diag_len, diag_len, rho_res)

    # Crear una matriz de acumulación
    accumulator = np.zeros((len(rhos), len(thetas)), dtype=np.uint64)

    # Encuentra píxeles de borde en la imagen
    edge_pixels = np.argwhere(imagen_umbral > 0)

    # Hough Transform
    for pixel in edge_pixels:
        x, y = pixel[1], pixel[0]
        for theta_idx, theta in enumerate(thetas):
            rho = int(x * np.cos(theta) + y * np.sin(theta))
            rho_idx = np.argmin(np.abs(rhos - rho))
            accumulator[rho_idx, theta_idx] += 1

    # Encontrar las líneas detectadas en la matriz de acumulación
    lines = []

    for rho_idx, theta_idx in np.argwhere(accumulator > threshold):
        rho = rhos[rho_idx]
        theta = thetas[theta_idx]
        lines.append((rho, theta))

    # Encontrar las intersecciones en la Transformada de Hough
    intersecciones = np.argwhere(accumulator >= threshold)

# Mostrar la transformada de Hough con las intersecciones marcadas
    plt.imshow(accumulator, cmap='jet')
    plt.title('Transformada de Hough')
    plt.show()


    # Filtrar líneas cercanas
    filtered_lines = []
    for line1 in lines:
        rho1, theta1 = line1
        keep = True
        for line2 in filtered_lines:
            rho2, theta2 = line2
            if np.abs(rho1 - rho2) < 30 and np.abs(theta1 - theta2) < np.deg2rad(10):
                keep = False
                break
        if keep:
            filtered_lines.append(line1)

    # Dibujar las líneas detectadas en la imagen original
    imagen_color = cv2.cvtColor(imagen, cv2.COLOR_GRAY2BGR)
    
    for rho, theta in filtered_lines:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        cv2.line(imagen_color, (x1, y1), (x2, y2), (0, 0, 255), 2)

    return imagen_color

# Ejemplo de uso de la función
imagen_con_lineas = detectar_lineas_hough('c0.png', theta_res=0.1, rho_res=1, threshold=50, umbral_bajo=50, umbral_alto=150)
plt.imshow(cv2.cvtColor(imagen_con_lineas, cv2.COLOR_BGR2RGB))
plt.title('Líneas Detectadas en la Carretera')
plt.show()

: 